本次作业以垃圾邮件分类任务为基础，要求提取文本特征并使用朴素贝叶斯算法进行垃圾邮件识别（调用已有工具包或自行实现）。

### 任务介绍
电子邮件是互联网的一项重要服务，在大家的学习、工作和生活中会广泛使用。但是大家的邮箱常常被各种各样的垃圾邮件填充了。有统计显示，每天互联网上产生的垃圾邮件有几百亿近千亿的量级。因此，对电子邮件服务提供商来说，垃圾邮件过滤是一项重要功能。而朴素贝叶斯算法在垃圾邮件识别任务上一直表现非常好，至今仍然有很多系统在使用朴素贝叶斯算法作为基本的垃圾邮件识别算法。

本次实验数据集来自[Trec06](https://plg.uwaterloo.ca/cgi-bin/cgiwrap/gvcormac/foo06)的中文垃圾邮件数据集，目录解压后包含三个文件夹，其中data目录下是所有的邮件（未分词），已分词好的邮件在data_cut目录下。邮件分为邮件头部分和正文部分，两部分之间一般有空行隔开。标签数据在label文件夹下，文件中每行是标签和对应的邮件路径。‘spam’表示垃圾邮件，‘ham’表示正常邮件。

本次实验

基本要求：
1. 提取正文部分的文本特征；
2. 划分训练集和测试集（可以借助工具包。一般笔记本就足够运行所有数据，认为实现困难或算力不够的同学可以采样一部分数据进行实验。）；
3. 使用朴素贝叶斯算法完成垃圾邮件的分类与预测，要求测试集准确率Accuracy、精准率Precision、召回率Recall均高于0.9（本次实验可以使用已有的一些工具包完成如sklearn）；
4. 对比特征数目（词表大小）对模型效果的影响；
5. 提交代码和实验报告。

扩展要求：
1. 邮件头信息有时也可以协助判断垃圾邮件，欢迎学有余力的同学们尝试；
2. 尝试自行实现朴素贝叶斯算法细节；
3. 尝试对比不同的概率计算方法。

### 导入工具包

In [5]:
'''
提示：
若调用已有工具包，sklearn中提供了一些可能会用到的类。
'''
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer # 提取文本特征向量的类
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, ComplementNB # 三种朴素贝叶斯算法，差别在于估计p(x|y)的方式
from sklearn.model_selection import train_test_split, GridSearchCV # 数据集划分和网格搜索交叉验证
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score # 评估指标
RANDOM_SEED = 2025 

In [16]:
import os
import numpy as np

data_dir = os.path.join(os.getcwd(),'trec06c-utf8') # 数据目录
print(os.getcwd())
# First, let's check what's in your current directory
print("Current working directory:", os.getcwd())
print("Contents:", os.listdir('.'))
label_file = os.path.join(data_dir,'label','index') # 标签文件路径

labels_array = [] # 存储标签
texts_array = [] #存储文本

with open(label_file, 'r') as f:
    lines = f.readlines()
    for line in lines:
        label,path = line.strip().split() # 分割标签和路径
        # parts = line.strip().split() # 分割标签和路径
        # label,path = parts[0],parts[1] # 获取标签和路径
        if label == 'spam':
            labels_array.append(1) # 将'spam'标签转换为1
        else:
            labels_array.append(0) # 将'ham'标签转换为0
        
        email_path = os.path.join(data_dir, path.replace('../data','data_cut')) # 构建邮件文件的完整路径
        with open(email_path,'r',encoding='utf-8',errors='ignore') as email_f:
            email_lines = email_f.readlines() # 读取邮件内容
            is_body = False # 标记是否在邮件正文部分
            email_text = [] # 存储邮件正文
            for email_line in email_lines:
                email_line = email_line.strip() # 去除行首尾空格
                if email_line == '': # 如果是空行，表示正文开始
                    is_body = True # 设置标记为True
                if is_body:
                    email_text += email_line.split() # 将正文行按空格分割并添加到email_text中
            texts_array.append(' '.join(email_text)) # 将正文列表转换为字符串并添加到texts_array中

labels = np.array(labels_array) # 转换为NumPy数组

d:\Computer_Science\御风\机器学习作业\贝叶斯垃圾邮件识别\hw3
Current working directory: d:\Computer_Science\御风\机器学习作业\贝叶斯垃圾邮件识别\hw3
Contents: ['hw3.html', 'hw3.ipynb', 'hw3.md', 'hw3.py', 'trec06c-utf8']


### 划分数据集

In [ ]:
# 划分数据集
train_texts, test_texts, train_labels, test_labels = train_test_split(texts_array, labels, test_size=0.2, random_state=RANDOM_SEED) # 划分数据集，80%训练集，20%测试集

# 从训练集中划分出10%作为验证集 (方便调参数)
train_indices, val_indices = train_test_split(np.arange(len(train_texts)),test_size=0.1, random_state=RANDOM_SEED) # 从训练集中划分出10%作为验证集

### 定义分类器模型

In [18]:
from typing import Union,Dict
class NaiveBayes:
    def __init__(self,max_df:Union[float,int]=1.0,min_df:Union[float,int]=1,tfidf:bool = False, type:str='multinomial') -> None:
        '''
        max_df: 特征向量的最大值，默认为1.0, 超过该阈值的词语会被过滤掉
        min_df: 特征向量的最小文档频率，默认为1, 低于该阈值的词语会被过滤掉
        tfidf: 是否使用TF-IDF特征向量，默认为False
        type: 朴素贝叶斯分类器的类型，默认为'multinomial',可选值有'multinomial', 'bernoulli', 'complement'
        '''
        self.max_df = max_df
        self.min_df = min_df
        self.tfidf = tfidf
        self.type = type

    def fit(self,X,y) -> None:
        '''
        X: 文本数据
        y: 标签
        '''
        if self.tfidf:
            self.vectorizer = TfidfVectorizer(max_df=self.max_df, min_df=self.min_df) # 使用TF-IDF特征向量
        else:
            self.vectorizer = CountVectorizer(max_df=self.max_df, min_df=self.min_df) # 使用词频特征向量
        
        X = self.vectorizer.fit_transform(X) # 将文本数据转换为特征向量
        print(X.shape) # 打印特征向量的形状
        if self.type == 'multinomial':
            self.model = MultinomialNB() # 使用多项式朴素贝叶斯分类器
        elif self.type == 'bernoulli':
            self.model = BernoulliNB() # 使用伯努利朴素贝叶斯分类器
        elif self.type == 'complement':
            self.model = ComplementNB() # 使用补充朴素贝叶斯分类器
        else:
            raise ValueError("Invalid type, must be one of 'multinomial', 'bernoulli', 'complement'") # 如果类型不合法，抛出异常
        self.model.fit(X, y) # 训练模型

    def predict(self,X) -> np.ndarray:
        '''
        X: 文本数据
        返回预测结果
        '''
        assert hasattr(self,'vectorizer'),'Please train the model first' # 确保模型已经训练
        assert hasattr(self, 'model'),'Please train the model first' # 确保模型已经训练
        X = self.vectorizer.transform(X) # 将文本数据转换为特征向量
        return self.model.predict(X) # 返回预测结果


    def get_params(self,deep:bool=True) -> Dict[str,Union[float,int,bool,str]]:
        '''
        返回模型参数
        deep: 是否深度拷贝，默认为True
        '''
        return {
            'max_df': self.max_df,
            'min_df': self.min_df,
            'tfidf': self.tfidf,
            'type': self.type
        }


    def set_params(self,**params) -> 'NaiveBayes':
        '''
        设置模型参数
        params: 参数字典
        返回当前对象
        '''
        for param,value in params.items():
            setattr(self, param, value) # 设置参数
        return self # 返回当前对象

### 训练和测试模型

In [ ]:
## Model multinomial Naive Bayes
model = NaiveBayes(type='multinomial',min_df=5,max_df=0.95) # 创建朴素贝叶斯分类器对象
# 至少出现5个词，最大出现频率为0.95
model.fit(train_texts,train_labels) # 训练模型
pred_test_labels = model.predict(test_texts) # 在测试集上进行预测
# Fix the print statement - remove curly braces and fix the f1_score call
print(f'data_cut')
print(f'Params: {model.get_params()}')
print(f'Accuracy: {accuracy_score(test_labels, pred_test_labels):.4f}')
print(f'Precision: {precision_score(test_labels, pred_test_labels):.4f}')
print(f'Recall: {recall_score(test_labels, pred_test_labels):.4f}')
print(f'F1: {f1_score(test_labels, pred_test_labels):.4f}')

(51696, 65642)
Params: {'max_df': 0.95, 'min_df': 5, 'tfidf': False, 'type': 'multinomial'}
Accuracy: 0.9759
Precision: 0.9810
Recall: 0.9825
F1: 0.9818


In [22]:
## Model Bernoulli Naive Bayes
model = NaiveBayes(type='bernoulli',min_df=5,max_df=0.95) # 创建朴素贝叶斯分类器对象
# 至少出现5个词，最大出现频率为0.95
model.fit(train_texts,train_labels) # 训练模型
pred_test_labels = model.predict(test_texts) # 在测试集上进行预测
# Fix the print statement - remove curly braces and fix the f1_score call
print(f'data_cut')
print(f'Params: {model.get_params()}')
print(f'Accuracy: {accuracy_score(test_labels, pred_test_labels):.4f}')
print(f'Precision: {precision_score(test_labels, pred_test_labels):.4f}')
print(f'Recall: {recall_score(test_labels, pred_test_labels):.4f}')
print(f'F1: {f1_score(test_labels, pred_test_labels):.4f}')

(51696, 65642)
data_cut
Params: {'max_df': 0.95, 'min_df': 5, 'tfidf': False, 'type': 'bernoulli'}
Accuracy: 0.9290
Precision: 0.9838
Recall: 0.9073
F1: 0.9440


In [24]:
## Model Complement Naive Bayes
model = NaiveBayes(type='complement',min_df=5,max_df=0.95) # 创建朴素贝叶斯分类器对象
# 至少出现5个词，最大出现频率为0.95
model.fit(train_texts,train_labels) # 训练模型
pred_test_labels = model.predict(test_texts) # 在测试集上进行预测
# Fix the print statement - remove curly braces and fix the f1_score call
print(f'data_cut')
print(f'Params: {model.get_params()}')
print(f'Accuracy: {accuracy_score(test_labels, pred_test_labels):.4f}')
print(f'Precision: {precision_score(test_labels, pred_test_labels):.4f}')
print(f'Recall: {recall_score(test_labels, pred_test_labels):.4f}')
print(f'F1: {f1_score(test_labels, pred_test_labels):.4f}')

(51696, 65642)
data_cut
Params: {'max_df': 0.95, 'min_df': 5, 'tfidf': False, 'type': 'complement'}
Accuracy: 0.9711
Precision: 0.9817
Recall: 0.9743
F1: 0.9780


### 调试超参数


In [26]:
params = {
    'max_df':[1.0,0.999,0.998,0.997,0.996,0.995,0.99],
    'min_df':range(1,7),
    'tfidf':[True,False],
    'type':['multinomial','bernoulli','complement']
}

grid_search = GridSearchCV(NaiveBayes(),params,cv=[(train_indices,val_indices)],scoring='f1',verbose=100,n_jobs=-1) # 网格搜索交叉验证
grid_search.fit(train_texts,train_labels) # 训练模型
print(f'Best params: {grid_search.best_params_}') # 打印最佳参数
print(f'Best score: {grid_search.best_score_:.4f}') # 打印最佳得分
best_model = NaiveBayes(**grid_search.best_params_) # 创建最佳模型
best_model.fit(train_texts, train_labels) # 训练最佳模型
pred_test_labels = best_model.predict(test_texts) # 在测试集上进行预测
print(f'Best model params: {best_model.get_params()}') # 打印最佳模型参数
print(f'Accuracy: {accuracy_score(test_labels, pred_test_labels):.4f}') # 打印准确率
print(f'Precision: {precision_score(test_labels, pred_test_labels):.4f}')
print(f'Recall: {recall_score(test_labels, pred_test_labels):.4f}') # 打印召回率
print(f'F1: {f1_score(test_labels, pred_test_labels):.4f}') # 打印F1分数


Fitting 1 folds for each of 252 candidates, totalling 252 fits
(51696, 183259)
Best params: {'max_df': 1.0, 'min_df': 1, 'tfidf': True, 'type': 'multinomial'}
Best score: 0.9853
(51696, 183259)
Best model params: {'max_df': 1.0, 'min_df': 1, 'tfidf': True, 'type': 'multinomial'}
Accuracy: 0.9802
Precision: 0.9788
Recall: 0.9914
F1: 0.9851


### 分析

#### 1. 实验总结

本次实验成功实现了基于朴素贝叶斯算法的中文垃圾邮件分类系统，主要完成了以下工作：

- **数据预处理**：从TREC06C中文垃圾邮件数据集中提取邮件正文，将文本转换为特征向量
- **模型对比**：测试了三种朴素贝叶斯变体（Multinomial、Bernoulli、Complement）
- **特征工程**：对比了词频特征(CountVectorizer)和TF-IDF特征的效果
- **超参数优化**：通过网格搜索找到最佳参数组合

#### 2. 关键发现

##### 2.1 朴素贝叶斯变体性能对比

通过实验对比发现：
- **Multinomial NB**: 在中文文本分类中表现优异，适合处理词频特征
- **Complement NB**: 在不平衡数据集上通常有更好的表现
- **Bernoulli NB**: 适合处理二值化特征，但在词频丰富的文本中可能不如Multinomial

##### 2.2 特征提取策略影响

- **TF-IDF vs 词频**: TF-IDF通过降低高频词权重，能更好地捕捉文档的区分性特征
- **词汇表大小**: `min_df`和`max_df`参数对性能有显著影响
  - `min_df=5`: 过滤掉出现频率过低的噪声词汇
  - `max_df=0.95`: 过滤掉过于常见的停用词

##### 2.3 性能指标分析

从实验结果来看：
- **准确率(Accuracy)**: 反映整体分类正确率
- **精确率(Precision)**: 衡量垃圾邮件识别的准确性，高精确率避免误杀正常邮件
- **召回率(Recall)**: 衡量垃圾邮件的捕获能力，高召回率确保垃圾邮件不漏检
- **F1分数**: 平衡精确率和召回率的综合指标

#### 3. 技术要点

##### 3.1 中文文本处理挑战
- **编码问题**: 使用`encoding='utf-8', errors='ignore'`处理多种编码格式
- **分词质量**: 数据集已预分词，避免了中文分词的复杂性
- **特征稀疏性**: 中文词汇量大，特征向量维度高且稀疏

##### 3.2 朴素贝叶斯优势
- **训练速度快**: 线性时间复杂度，适合大规模文本数据
- **内存效率高**: 只需存储词汇概率，模型紧凑
- **可解释性强**: 可以分析关键词的重要性
- **鲁棒性好**: 对噪声数据不敏感

#### 4. 实际应用价值

##### 4.1 工程实践意义
- **实时性**: 朴素贝叶斯预测速度快，满足邮件系统实时过滤需求
- **可扩展性**: 易于增量学习新的垃圾邮件模式
- **部署简单**: 模型轻量，便于集成到邮件服务器

##### 4.2 性能评估
目标要求Accuracy、Precision、Recall均高于0.9，通过合理的参数调优，朴素贝叶斯算法能够达到这一性能标准。

#### 5. 改进方向

##### 5.1 特征工程优化
- **N-gram特征**: 考虑词语组合信息，如bigram、trigram
- **邮件头特征**: 利用发件人、主题等元信息
- **文本长度特征**: 垃圾邮件通常有特定的长度分布模式

##### 5.2 模型集成
- **投票机制**: 结合多个朴素贝叶斯变体的预测结果
- **混合模型**: 与SVM、随机森林等算法组合
- **深度学习**: 探索BERT等预训练模型在垃圾邮件检测中的应用

#### 6. 结论

朴素贝叶斯算法在中文垃圾邮件识别任务中表现出色，具有以下特点：

✅ **高效性**: 训练和预测速度快，适合实时应用  
✅ **准确性**: 通过参数优化可达到较高的分类精度  
✅ **实用性**: 模型简单，易于理解和部署  
✅ **稳定性**: 对参数变化不敏感，泛化能力强  

这验证了朴素贝叶斯算法在文本分类领域的经典地位，为垃圾邮件过滤系统提供了可靠的技术基础。
